# Overview

This script implements an end-to-end data transformation and consolidation pipeline for flight data stored in CSV files. It addresses issues such as inconsistent data types, duplicate rows, and formatting irregularities, ultimately unifying the data into a single, reliable table in a DuckDB database.

## What it does

- **Database Setup & CSV Import:**  
  Connects to (or creates) a DuckDB database file and imports CSV files from the "helium" folder. Each CSV file is converted into a table with its name derived from the filename.

- **Data Consolidation:**  
  Merges individual CSV tables into a single "flights" table using a UNION ALL query, making the dataset easier to query and analyze.

- **Deduplication & Data Cleaning:**  
  Identifies and removes duplicate rows to ensure data integrity. It also cleans string columns by removing extraneous characters and standardizing empty values.

- **Schema Transformation:**  
  Updates the schema by converting certain columns to appropriate data types (e.g., from VARCHAR to INT) and dropping unnecessary columns. This step standardizes the structure of the data.

## Why it does it

- **Consistency:**  
  The script handles inconsistencies in the data by enforcing uniform data types and standardized formatting, which is essential for reliable analysis.

- **Efficiency:**  
  Automating the import, cleaning, and consolidation processes reduces the need for manual intervention and minimizes the risk of human error.

- **Data Integrity:**  
  Deduplication and careful data cleaning ensure that the final consolidated dataset is accurate and ready for further analysis.

- **Scalability:**  
  By merging multiple CSV sources into one unified table, the pipeline simplifies data management and makes the dataset easier to query at scale.


In [11]:
import duckdb
import glob
import os
import pandas as pd

# Overview

This script creates, if it doesn't exist, a DuckDB database file named "flight_data.duckdb" and automatically imports all CSV files from the "helium" folder. Each CSV file is turned into a table in the database, with the table name derived from the CSV filename.

## What it does

- **Database connection:**  
  Connects to the DuckDB database file, creating it if necessary, so you don't have to worry about setting up the database manually.

- **CSV file discovery:**  
  Uses glob to find all CSV files in the "helium" folder, ensuring every file is on deck for import.

- **Table creation:**  
  For each CSV file, it:
  - Extracts the filename (without the extension) to use as the table name.
  - Reads the CSV file with `read_csv_auto`, forcing all values to be treated as strings.
  - Creates a new table in DuckDB based on the CSV content.

- **Progress Feedback:**  
  Prints a confirmation message for each table created, so you're never left in the dark about what's been processed.

- **Cleanup:**  
  Closes the database connection once all files have been imported, keeping everything neat and tidy.

## Why it does it

Because manually importing CSV files into a database is for those who enjoy unnecessary hassle. This script automates the import process, ensuring your flight data is consistently and reliably loaded into DuckDB, freeing you up to focus on listening to Alisson Becker's chant over and over again because he saved Liverpool against PSG, let's be honest.


In [2]:
# Connect to (or create) the DuckDB database file named flight_data.duckdb
con = duckdb.connect("flight_data.duckdb")

# Define the folder containing CSV files
csv_folder = "helium"

# Find all CSV files in the folder
csv_files = glob.glob(os.path.join(csv_folder, "*.csv"))

# Loop over each CSV file and create a table from it
for csv_file in csv_files:
    # Use the file name (without extension) as the table name
    table_name = os.path.splitext(os.path.basename(csv_file))[0]
    
    # Create the table by reading the CSV file and converting all values to strings
    query = f"""
    CREATE TABLE {table_name} AS
    SELECT * FROM read_csv_auto('{csv_file}', all_varchar=True);
    """
    con.execute(query)
    print(f"Table '{table_name}' created from '{csv_file}'.")

# Close the connection
con.close()

Table 'VRA_2002_02' created from 'helium/VRA_2002_02.csv'.
Table 'VRA_2006_03' created from 'helium/VRA_2006_03.csv'.
Table 'VRA_2001_12' created from 'helium/VRA_2001_12.csv'.
Table 'VRA_2023_02' created from 'helium/VRA_2023_02.csv'.
Table 'VRA_2004_11' created from 'helium/VRA_2004_11.csv'.
Table 'VRA_2008_06' created from 'helium/VRA_2008_06.csv'.
Table 'VRA_2004_05' created from 'helium/VRA_2004_05.csv'.
Table 'VRA_2010_06' created from 'helium/VRA_2010_06.csv'.
Table 'VRA_2024_04' created from 'helium/VRA_2024_04.csv'.
Table 'VRA_2014_09' created from 'helium/VRA_2014_09.csv'.
Table 'VRA_2006_05' created from 'helium/VRA_2006_05.csv'.
Table 'VRA_2016_10' created from 'helium/VRA_2016_10.csv'.
Table 'VRA_2007_02' created from 'helium/VRA_2007_02.csv'.
Table 'VRA_2022_04' created from 'helium/VRA_2022_04.csv'.
Table 'VRA_2006_11' created from 'helium/VRA_2006_11.csv'.
Table 'VRA_2020_02' created from 'helium/VRA_2020_02.csv'.
Table 'VRA_2009_01' created from 'helium/VRA_2009_01.csv

In [6]:
# Connect to the existing DuckDB database
con = duckdb.connect("flight_data.duckdb")

# Get a list of all user-created tables in the main schema
# (Excluding system tables and ignoring an existing 'flights' table, if any)
tables = con.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'main'
      AND table_name NOT IN ('flights')
""").fetchall()

# Build a UNION ALL query to merge all tables into one
# We'll create (or replace) a table named 'flights'
union_parts = []
for (table_name,) in tables:
    union_parts.append(f"SELECT * FROM {table_name}")

# If no tables were found, you might want to handle that separately
if not union_parts:
    print("No user tables found to merge.")
else:
    union_query = """
        CREATE OR REPLACE TABLE flights AS
        {}
    """.format("\nUNION ALL\n".join(union_parts))

    # Execute the query to merge all tables into 'flights'
    con.execute(union_query)

    # Count how many rows ended up in the new 'flights' table
    row_count = con.execute("SELECT COUNT(*) FROM flights;").fetchone()[0]
    print(f"Number of rows in 'flights' table: {row_count}")

# Close the connection
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Number of rows in 'flights' table: 24315692


In [7]:
# Connect to your DuckDB database
con = duckdb.connect("flight_monolith.duckdb")

# 1. Count total rows
total_rows = con.execute("SELECT COUNT(*) FROM flights;").fetchone()[0]

# 2. Count distinct rows
distinct_rows = con.execute("""
    SELECT COUNT(*) 
    FROM (SELECT DISTINCT * FROM flights) AS temp
""").fetchone()[0]

duplicates = total_rows - distinct_rows

print(f"Total rows: {total_rows}")
print(f"Distinct rows: {distinct_rows}")
print(f"Duplicates: {duplicates}")

# 3. Remove duplicates by recreating 'flights' with DISTINCT rows only
con.execute("""
CREATE OR REPLACE TABLE flights AS
SELECT DISTINCT * 
FROM flights;
""")

# Verify the new row count after deduplication
new_total_rows = con.execute("SELECT COUNT(*) FROM flights;").fetchone()[0]
print(f"New total rows after removing duplicates: {new_total_rows}")

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Total rows: 24315692
Distinct rows: 24288657
Duplicates: 27035


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

New total rows after removing duplicates: 24288657


In [9]:
# Connect to the DuckDB database
con = duckdb.connect("flight_monolith.duckdb")

# Get column info from the 'flights' table
schema_info = con.execute("PRAGMA table_info('flights');").fetchall()

# Loop over each column and update only string-type columns
for col_index, col_name, col_type, *rest in schema_info:
    if col_type.upper().startswith("VARCHAR") or col_type.upper().startswith("STRING"):
        # Replace sequences of quotes with an empty string, then use NULLIF to set it to NULL if empty.
        # Note the regex pattern '[''"]+' uses doubled single quotes to represent a literal single quote.
        update_query = f"""
            UPDATE flights
            SET "{col_name}" = NULLIF(regexp_replace("{col_name}", '[''"]+', ''), '')
        """
        con.execute(update_query)
        print(f"Cleaned column '{col_name}'.")

# Optionally, check the results (for example, view the first few rows)
result = con.execute("SELECT * FROM flights LIMIT 10;").fetchall()
print("Sample rows after cleaning:")
for row in result:
    print(row)

con.close()

Cleaned column 'empresa_aerea'.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cleaned column 'long_empresa_aerea'.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cleaned column 'numero_voo'.
Cleaned column 'codigo_DI'.
Cleaned column 'codigo_tipo_linha'.
Cleaned column 'modelo_equipamento'.
Cleaned column 'numero_assentos'.
Cleaned column 'aeroporto_origem'.


...

Cleaned column 'descricao_aeroporto_destino'.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cleaned column 'chegada_prevista'.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cleaned column 'chegada_real'.
Cleaned column 'situacao_voo'.
Cleaned column 'justificativa'.
Cleaned column 'referencia'.
Cleaned column 'situacao_partida'.
Cleaned column 'situacao_chegada'.
Cleaned column 'ano'.
Cleaned column 'mes'.
Sample rows after cleaning:
('VRG', None, '2206', '0', 'N', None, None, 'SBGL', None, '19/02/2001 19:00', None, 'SBBR', None, '19/02/2001 20:31', None, 'REALIZADO', 'N/A', None, None, None, '2001', '02')
('TAM', None, '3941', '0', 'N', None, None, 'SBGL', None, '20/02/2001 16:45', None, 'SBGR', None, '20/02/2001 17:45', None, 'REALIZADO', 'N/A', None, None, None, '2001', '02')
('VRG', None, '2261', '0', 'N', None, None, 'SBSL', None, '20/02/2001 11:10', None, 'SBFZ', None, '20/02/2001 12:25', None, 'REALIZADO', 'N/A', None, None, None, '2001', '02')
('RSL', None, '5373', '0', 'E', None, None, 'SBBH', None, '20/02/2001 09:34', None, 'SBRJ', None, '20/02/2001 10:21', None, 'REALIZADO', 'N/A', None, None, None, '2001', '02')
('RSL', None, '5000', '0', 'E',

In [10]:
# Connect to your DuckDB database
con = duckdb.connect("flight_monolith.duckdb")

# Retrieve schema information from the 'flights' table
schema_info = con.execute("PRAGMA table_info('flights');").fetchall()

# Extract the column names from the schema info
# Assuming all columns are string columns (loaded as VARCHAR/STRING)
columns = [col[1] for col in schema_info]

# Construct a WHERE condition that checks if any column is an empty string
# This will generate a condition like: "col1" = '' OR "col2" = '' OR ...
condition = " OR ".join([f'"{col}" = \'\' ' for col in columns])

# Build the full SQL query to count the rows that have at least one empty string
query = f"SELECT COUNT(*) FROM flights WHERE {condition};"

# Execute the query and fetch the result
empty_rows_count = con.execute(query).fetchone()[0]
print(f"Number of rows with at least one empty string: {empty_rows_count}")

con.close()

Number of rows with at least one empty string: 0


In [12]:
# Connect to the DuckDB database
con = duckdb.connect("flight_monolith.duckdb")

# Retrieve schema information using PRAGMA table_info, which includes:
# cid, name, type, notnull, dflt_value, and pk for each column
schema_df = con.execute("PRAGMA table_info('flights');").fetchdf()

print("Schema of the 'flights' table:")
print(schema_df)

# Count the number of rows in the flights table
row_count = con.execute("SELECT COUNT(*) FROM flights;").fetchone()[0]

# The number of columns is the number of rows in the schema_df DataFrame
num_columns = schema_df.shape[0]

print(f"\nNumber of rows: {row_count}")
print(f"Number of columns: {num_columns}")

# Close the connection
con.close()

Schema of the 'flights' table:
    cid                         name     type  notnull dflt_value     pk
0     0                empresa_aerea  VARCHAR    False       None  False
1     1           long_empresa_aerea  VARCHAR    False       None  False
2     2                   numero_voo  VARCHAR    False       None  False
3     3                    codigo_DI  VARCHAR    False       None  False
4     4            codigo_tipo_linha  VARCHAR    False       None  False
5     5           modelo_equipamento  VARCHAR    False       None  False
6     6              numero_assentos  VARCHAR    False       None  False
7     7             aeroporto_origem  VARCHAR    False       None  False
8     8   descricao_aeroporto_origem  VARCHAR    False       None  False
9     9             partida_prevista  VARCHAR    False       None  False
10   10                 partida_real  VARCHAR    False       None  False
11   11            aeroporto_destino  VARCHAR    False       None  False
12   12  descricao_a

In [13]:
# Connect to your DuckDB database
con = duckdb.connect("flight_monolith.duckdb")

# Alter each column from VARCHAR to INT
# The USING clause ensures DuckDB knows how to transform the existing data.
con.execute("""
    ALTER TABLE flights 
    ALTER COLUMN numero_assentos 
    TYPE INT 
    USING CAST(numero_assentos AS INT)
""")

con.execute("""
    ALTER TABLE flights 
    ALTER COLUMN ano 
    TYPE INT 
    USING CAST(ano AS INT)
""")

con.execute("""
    ALTER TABLE flights 
    ALTER COLUMN mes 
    TYPE INT 
    USING CAST(mes AS INT)
""")

# (Optional) Show the updated schema
updated_schema = con.execute("PRAGMA table_info('flights');").fetchdf()
print("Updated Schema of the 'flights' table:")
print(updated_schema)

con.close()

Updated Schema of the 'flights' table:
    cid                         name     type  notnull dflt_value     pk
0     0                empresa_aerea  VARCHAR    False       None  False
1     1           long_empresa_aerea  VARCHAR    False       None  False
2     2                   numero_voo  VARCHAR    False       None  False
3     3                    codigo_DI  VARCHAR    False       None  False
4     4            codigo_tipo_linha  VARCHAR    False       None  False
5     5           modelo_equipamento  VARCHAR    False       None  False
6     6              numero_assentos  INTEGER    False       None  False
7     7             aeroporto_origem  VARCHAR    False       None  False
8     8   descricao_aeroporto_origem  VARCHAR    False       None  False
9     9             partida_prevista  VARCHAR    False       None  False
10   10                 partida_real  VARCHAR    False       None  False
11   11            aeroporto_destino  VARCHAR    False       None  False
12   12  des

In [14]:
# Connect to your DuckDB database
con = duckdb.connect("flight_monolith.duckdb")

# Drop the 'referencia' column from the 'flights' table
con.execute("ALTER TABLE flights DROP COLUMN referencia;")

# (Optional) Verify that the column was removed
updated_schema = con.execute("PRAGMA table_info('flights');").fetchdf()
print("Updated schema of the 'flights' table after dropping 'referencia':")
print(updated_schema)

con.close()

Updated schema of the 'flights' table after dropping 'referencia':
    cid                         name     type  notnull dflt_value     pk
0     0                empresa_aerea  VARCHAR    False       None  False
1     1           long_empresa_aerea  VARCHAR    False       None  False
2     2                   numero_voo  VARCHAR    False       None  False
3     3                    codigo_DI  VARCHAR    False       None  False
4     4            codigo_tipo_linha  VARCHAR    False       None  False
5     5           modelo_equipamento  VARCHAR    False       None  False
6     6              numero_assentos  INTEGER    False       None  False
7     7             aeroporto_origem  VARCHAR    False       None  False
8     8   descricao_aeroporto_origem  VARCHAR    False       None  False
9     9             partida_prevista  VARCHAR    False       None  False
10   10                 partida_real  VARCHAR    False       None  False
11   11            aeroporto_destino  VARCHAR    False   

In [15]:
con = duckdb.connect("flight_monolith.duckdb")

# 1. Get the current column list
schema_info = con.execute("PRAGMA table_info('flights');").fetchall()
# schema_info is a list of tuples: (cid, name, type, notnull, dflt_value, pk)
all_columns = [row[1] for row in schema_info]  # Extract just the column names

# 2. Build a new column order: 'ano' first, 'mes' second, then the rest
#    (Remove them first so we don't duplicate)
all_columns.remove("ano")
all_columns.remove("mes")
new_order = ["ano", "mes"] + all_columns

# 3. Create a temporary table in the new order
#    We'll quote column names in case they have special characters.
select_clause = ", ".join([f'"{col}"' for col in new_order])
create_temp_table = f"""
    CREATE TABLE flights_temp AS
    SELECT {select_clause}
    FROM flights
"""
con.execute(create_temp_table)

# 4. Drop the old flights table
con.execute("DROP TABLE flights")

# 5. Rename the temporary table to flights
con.execute("ALTER TABLE flights_temp RENAME TO flights")

# (Optional) Show the updated schema
updated_schema = con.execute("PRAGMA table_info('flights');").fetchdf()
print("Updated schema of the 'flights' table:")
print(updated_schema)

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Updated schema of the 'flights' table:
    cid                         name     type  notnull dflt_value     pk
0     0                          ano  INTEGER    False       None  False
1     1                          mes  INTEGER    False       None  False
2     2                empresa_aerea  VARCHAR    False       None  False
3     3           long_empresa_aerea  VARCHAR    False       None  False
4     4                   numero_voo  VARCHAR    False       None  False
5     5                    codigo_DI  VARCHAR    False       None  False
6     6            codigo_tipo_linha  VARCHAR    False       None  False
7     7           modelo_equipamento  VARCHAR    False       None  False
8     8              numero_assentos  INTEGER    False       None  False
9     9             aeroporto_origem  VARCHAR    False       None  False
10   10   descricao_aeroporto_origem  VARCHAR    False       None  False
11   11             partida_prevista  VARCHAR    False       None  False
12   12     

In [16]:
# Connect to your DuckDB database
con = duckdb.connect("flight_monolith.duckdb")

# Create a new temporary table ordered by ano and mes
con.execute("""
    CREATE TABLE flights_temp AS
    SELECT * FROM flights
    ORDER BY ano, mes;
""")

# Drop the old flights table
con.execute("DROP TABLE flights;")

# Rename the temporary table to flights
con.execute("ALTER TABLE flights_temp RENAME TO flights;")

print("Table 'flights' has been reordered by ano and mes.")

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Table 'flights' has been reordered by ano and mes.


In [19]:
# Connect to your DuckDB database
con = duckdb.connect("flight_monolith.duckdb")

# 1. Get the first 10 rows as a DataFrame
df_first_10 = con.execute("SELECT * FROM flights LIMIT 10;").fetchdf()
print("First 10 rows:")
print(df_first_10)

# 2. Get 10 random rows as a DataFrame
df_random_10 = con.execute("SELECT * FROM flights ORDER BY RANDOM() LIMIT 10;").fetchdf()
print("\nRandom 10 rows:")
print(df_random_10)

# Close the connection
con.close()

First 10 rows:
    ano  mes empresa_aerea long_empresa_aerea numero_voo codigo_DI  \
0  2000    1           TBA               None       1690         0   
1  2000    1           BLC               None       8230         0   
2  2000    1           RSL               None       4100         0   
3  2000    1           TAB               None       1620         0   
4  2000    1           RLE               None       1070         0   
5  2000    1           PEP               None       2900         0   
6  2000    1           VRG               None       3260         0   
7  2000    1           TBA               None       2000         0   
8  2000    1           TBA               None       3120         0   
9  2000    1           BLC               None        171         9   

  codigo_tipo_linha modelo_equipamento  numero_assentos aeroporto_origem  ...  \
0                 N               None             <NA>             SBSV  ...   
1                 N               None             <

In [22]:
con = duckdb.connect("flight_monolith.duckdb")

# Add new columns one by one
con.execute("ALTER TABLE flights ADD COLUMN dia_partida_prevista DATE;")
con.execute("ALTER TABLE flights ADD COLUMN hora_partida_prevista TIME;")
con.execute("ALTER TABLE flights ADD COLUMN dia_partida_real DATE;")
con.execute("ALTER TABLE flights ADD COLUMN hora_partida_real TIME;")
con.execute("ALTER TABLE flights ADD COLUMN dia_chegada_prevista DATE;")
con.execute("ALTER TABLE flights ADD COLUMN hora_chegada_prevista TIME;")
con.execute("ALTER TABLE flights ADD COLUMN dia_chegada_real DATE;")
con.execute("ALTER TABLE flights ADD COLUMN hora_chegada_real TIME;")

# Update for partida_prevista: extract date and time parts
con.execute("""
    UPDATE flights
    SET 
        dia_partida_prevista = CASE
            WHEN length(partida_prevista) >= 10
                THEN CAST(strptime(substr(partida_prevista, 1, 10), '%d/%m/%Y') AS DATE)
            ELSE NULL
        END,
        hora_partida_prevista = CASE
            WHEN length(partida_prevista) > 10
                THEN CAST(strptime(substr(partida_prevista, 12, 5), '%H:%M') AS TIME)
            ELSE NULL
        END
    WHERE partida_prevista IS NOT NULL
""")

# Update for partida_real: extract date and time parts
con.execute("""
    UPDATE flights
    SET 
        dia_partida_real = CASE
            WHEN length(partida_real) >= 10
                THEN CAST(strptime(substr(partida_real, 1, 10), '%d/%m/%Y') AS DATE)
            ELSE NULL
        END,
        hora_partida_real = CASE
            WHEN length(partida_real) > 10
                THEN CAST(strptime(substr(partida_real, 12, 5), '%H:%M') AS TIME)
            ELSE NULL
        END
    WHERE partida_real IS NOT NULL
""")

# Update for chegada_prevista: extract date and time parts
con.execute("""
    UPDATE flights
    SET 
        dia_chegada_prevista = CASE
            WHEN length(chegada_prevista) >= 10
                THEN CAST(strptime(substr(chegada_prevista, 1, 10), '%d/%m/%Y') AS DATE)
            ELSE NULL
        END,
        hora_chegada_prevista = CASE
            WHEN length(chegada_prevista) > 10
                THEN CAST(strptime(substr(chegada_prevista, 12, 5), '%H:%M') AS TIME)
            ELSE NULL
        END
    WHERE chegada_prevista IS NOT NULL
""")

# Update for chegada_real: extract date and time parts
con.execute("""
    UPDATE flights
    SET 
        dia_chegada_real = CASE
            WHEN length(chegada_real) >= 10
                THEN CAST(strptime(substr(chegada_real, 1, 10), '%d/%m/%Y') AS DATE)
            ELSE NULL
        END,
        hora_chegada_real = CASE
            WHEN length(chegada_real) > 10
                THEN CAST(strptime(substr(chegada_real, 12, 5), '%H:%M') AS TIME)
            ELSE NULL
        END
    WHERE chegada_real IS NOT NULL
""")

# Verify the results by selecting a few rows
df_check = con.execute("""
    SELECT partida_prevista, dia_partida_prevista, hora_partida_prevista,
           partida_real,    dia_partida_real,    hora_partida_real,
           chegada_prevista, dia_chegada_prevista, hora_chegada_prevista,
           chegada_real,    dia_chegada_real,    hora_chegada_real
    FROM flights
    LIMIT 10
""").fetchdf()

print(df_check)

con.close()

   partida_prevista dia_partida_prevista hora_partida_prevista  \
0  01/01/2000 22:20           2000-01-01              22:20:00   
1  01/01/2000 18:20           2000-01-01              18:20:00   
2  01/01/2000 07:52           2000-01-01              07:52:00   
3  01/01/2000 09:00           2000-01-01              09:00:00   
4  01/01/2000 11:10           2000-01-01              11:10:00   
5  01/01/2000 10:30           2000-01-01              10:30:00   
6  01/01/2000 20:15           2000-01-01              20:15:00   
7  01/01/2000 12:10           2000-01-01              12:10:00   
8  01/01/2000 10:44           2000-01-01              10:44:00   
9              None                  NaT                  <NA>   

       partida_real dia_partida_real hora_partida_real  chegada_prevista  \
0              None              NaT              <NA>  02/01/2000 00:11   
1              None              NaT              <NA>  01/01/2000 19:00   
2              None              NaT         

In [25]:
con = duckdb.connect("flight_monolith.duckdb")

# 1) Add two new columns for the status
con.execute("ALTER TABLE flights ADD COLUMN status_partida VARCHAR;")
con.execute("ALTER TABLE flights ADD COLUMN status_chegada VARCHAR;")

# 2) Update status_partida
#    - We first check if any required columns are NULL (unknown).
#    - Otherwise we compare (date + time) for real vs. scheduled.
con.execute("""
    UPDATE flights
    SET status_partida = CASE
        WHEN dia_partida_prevista IS NULL OR hora_partida_prevista IS NULL
             OR dia_partida_real IS NULL OR hora_partida_real IS NULL
             THEN 'indefinido'
        WHEN (dia_partida_real + hora_partida_real) > (dia_partida_prevista + hora_partida_prevista)
             THEN 'atrasado'
        WHEN (dia_partida_real + hora_partida_real) < (dia_partida_prevista + hora_partida_prevista)
             THEN 'adiantado'
        ELSE 'ok'
    END
""")

# 3) Update status_chegada
con.execute("""
    UPDATE flights
    SET status_chegada = CASE
        WHEN dia_chegada_prevista IS NULL OR hora_chegada_prevista IS NULL
             OR dia_chegada_real IS NULL OR hora_chegada_real IS NULL
             THEN 'indefinido'
        WHEN (dia_chegada_real + hora_chegada_real) > (dia_chegada_prevista + hora_chegada_prevista)
             THEN 'atrasado'
        WHEN (dia_chegada_real + hora_chegada_real) < (dia_chegada_prevista + hora_chegada_prevista)
             THEN 'adiantado'
        ELSE 'ok'
    END
""")

# (Optional) Verify a few rows
df_check = con.execute("""
    SELECT dia_partida_prevista, hora_partida_prevista,
           dia_partida_real, hora_partida_real,
           status_partida,
           dia_chegada_prevista, hora_chegada_prevista,
           dia_chegada_real, hora_chegada_real,
           status_chegada
    FROM flights
    LIMIT 10
""").fetchdf()

print(df_check)

con.close()

  dia_partida_prevista hora_partida_prevista dia_partida_real  \
0           2000-01-01              22:20:00              NaT   
1           2000-01-01              18:20:00              NaT   
2           2000-01-01              07:52:00              NaT   
3           2000-01-01              09:00:00              NaT   
4           2000-01-01              11:10:00              NaT   
5           2000-01-01              10:30:00              NaT   
6           2000-01-01              20:15:00              NaT   
7           2000-01-01              12:10:00              NaT   
8           2000-01-01              10:44:00              NaT   
9                  NaT                  <NA>       2000-01-01   

  hora_partida_real status_partida dia_chegada_prevista hora_chegada_prevista  \
0              <NA>     indefinido           2000-01-02              00:11:00   
1              <NA>     indefinido           2000-01-01              19:00:00   
2              <NA>     indefinido       

In [26]:
con = duckdb.connect("flight_monolith.duckdb")

# (Optional) Verify a few rows
df_check = con.execute("""
    SELECT dia_partida_prevista, hora_partida_prevista,
           dia_partida_real, hora_partida_real,
           status_partida,
           dia_chegada_prevista, hora_chegada_prevista,
           dia_chegada_real, hora_chegada_real,
           status_chegada
    FROM flights
    LIMIT 10 offset 10000000
""").fetchdf()

print(df_check)

con.close()

  dia_partida_prevista hora_partida_prevista dia_partida_real  \
0           2011-02-14              09:09:00       2011-02-14   
1           2011-02-14              06:05:00       2011-02-14   
2           2011-02-15              07:03:00              NaT   
3           2011-02-15              15:40:00              NaT   
4           2011-02-15              16:50:00       2011-02-15   
5           2011-02-16              10:00:00       2011-02-16   
6           2011-02-17              13:58:00       2011-02-17   
7           2011-02-17              20:10:00       2011-02-17   
8           2011-02-17              03:00:00       2011-02-17   
9           2011-02-17              05:20:00       2011-02-17   

  hora_partida_real status_partida dia_chegada_prevista hora_chegada_prevista  \
0          09:09:00             ok           2011-02-14              10:22:00   
1          06:05:00             ok           2011-02-14              09:50:00   
2              <NA>     indefinido       

In [27]:
# Connect to the DuckDB database
con = duckdb.connect("flight_monolith.duckdb")

# Retrieve schema information using PRAGMA table_info, which includes:
# cid, name, type, notnull, dflt_value, and pk for each column
schema_df = con.execute("PRAGMA table_info('flights');").fetchdf()

print("Schema of the 'flights' table:")
print(schema_df)

# Count the number of rows in the flights table
row_count = con.execute("SELECT COUNT(*) FROM flights;").fetchone()[0]

# The number of columns is the number of rows in the schema_df DataFrame
num_columns = schema_df.shape[0]

print(f"\nNumber of rows: {row_count}")
print(f"Number of columns: {num_columns}")

# Close the connection
con.close()

Schema of the 'flights' table:
    cid                         name       type  notnull dflt_value     pk
0     0                          ano    INTEGER    False       None  False
1     1                          mes    INTEGER    False       None  False
2     2                empresa_aerea    VARCHAR    False       None  False
3     3           long_empresa_aerea    VARCHAR    False       None  False
4     4                   numero_voo    VARCHAR    False       None  False
5     5                    codigo_DI    VARCHAR    False       None  False
6     6            codigo_tipo_linha    VARCHAR    False       None  False
7     7           modelo_equipamento    VARCHAR    False       None  False
8     8              numero_assentos    INTEGER    False       None  False
9     9             aeroporto_origem    VARCHAR    False       None  False
10   10   descricao_aeroporto_origem    VARCHAR    False       None  False
11   11             partida_prevista    VARCHAR    False       None  